In [1]:
import os
import pandas as pd 
import numpy as np 

# visualisation
import matplotlib.pylab as plt
import matplotlib.font_manager as fm

# our scripts 
import src.processing as sp 
import settings as st
import utils.helpers as hp
import utils.imaging as ui

### Correct for the locations of the images 

In [2]:
# dr5_desc = sp.correct_location('gz_decals_volunteers_5', save = True, filename='dr5_votes')

### The data with the votes

In [3]:
dr5 = hp.read_parquet(st.data_dir, 'descriptions/dr5_votes')

### Generate the labels 

In [5]:
labels = sp.generate_labels('dr5_votes', save = False)

In [6]:
labels.head()

,iauname,png_loc,smooth-or-featured_smooth_fraction,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_artifact_fraction,disk-edge-on_yes_fraction,disk-edge-on_no_fraction,has-spiral-arms_yes_fraction,has-spiral-arms_no_fraction,bar_strong_fraction,...,spiral-arm-count_1_fraction,spiral-arm-count_2_fraction,spiral-arm-count_3_fraction,spiral-arm-count_4_fraction,spiral-arm-count_more-than-4_fraction,spiral-arm-count_cant-tell_fraction,merging_none_fraction,merging_minor-disturbance_fraction,merging_major-disturbance_fraction,merging_merger_fraction
0,J112953.88-000427.4,J112/J112953.88-000427.4.png,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,J104325.29+190335.0,J104/J104325.29+190335.0.png,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0,0.0,0.0,0.0
2,J104629.54+115415.1,J104/J104629.54+115415.1.png,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,J082950.68+125621.8,J082/J082950.68+125621.8.png,0.0,1.0,0.0,1.0,0.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,1.0,0.0,0.0,0.0
4,J122056.00-015022.0,J122/J122056.00-015022.0.png,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
